# Transfer learning with PyTorch
We're going to train a neural network to classify dogs and cats.

## Init, helpers, utils, ...

In [4]:
from pprint import pprint
import random
import datetime
import time

from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import os

import torchvision
from torchvision.datasets.folder import ImageFolder, default_loader

%matplotlib inline

from torchvision import transforms



In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Transforms

In [6]:
from torchvision import transforms

IMG_SIZE = 224  #224  #defined by NN model input
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_trans = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),  #256  #(IMG_SIZE, IMG_SIZE)  # some images are pretty small
    #transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_trans = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),  #256  #(IMG_SIZE, IMG_SIZE)
    #transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

## Dataset

In [8]:
BATCH_SIZE = 128  #2  #256  #512  #32  #220 for resnet152 on Dell Presison 5520 laptop, 400 for resnet18

n_classes = 2

from torch.utils.data import DataLoader

# use trained model to predict a pair of people in a new image

# test data set
test_ds = ImageFolder("data/raw/DUI/test", transform=val_trans, loader=default_loader)
print(f'len(test_ds) = {len(test_ds)}. ')

test_dl = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
)

len(test_ds) = 1421. 


# Predict with Trained Model

Pickle the trained model and predict image with it. 

In [2]:
# from one origin image to one canvas and predicted people boundaries

from predict_boxes import predict_boxes 

weight_name = 'network/weight/pose_model.pth'
# test_image = 'readme/ski.jpg'

# canvas, all_predicts = predict_boxes(test_image, weight_name)

Bulding VGG19


/home/tuo/anaconda3/envs/torch36/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:24: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [ ]:
# load the trained model
from network.ResNet_ronorigin import *
model = resnet_18()

for param in model.parameters():
    param.requires_grad = True
    
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model = model.to(device)

model.load_state_dict(torch.load(model_weights_path))

In [ ]:
# predict, WIP

# Eval
model.eval()  # IMPORTANT

with torch.no_grad():  # IMPORTANT
    for image_batch in imagefolder:
        
        # image_batch to single image
        canvas, all_predicts = predict_boxes(image_batch, weight_name)
        
        # canvas to crop
        pred = model(canvas, crop)
        
        # link and visualize
        
        # cv2.rectangle
        # cv2.curve